# 8.1 Fundamental concepts of `ngsxfem`

This unit gives a basic introduction to a few fundamental concepts in `ngsxfem`. Especially, we treat:

* level set geometries and its (P1) approximation (for higher order approximations see [intlset.ipynb](intlset.ipynb))
* Visualization of discontinuous (across level set zeros) functions
* handling of cut configurations inside the mesh (active elements / facets / dofs)
* Restriction of `FESpace`s, `integrators` and `Matrices` to active parts.

## Geometry description with the level set function

Most features of the `ngsxfem` add-on to NGSolve are based on a level set description.

Let $\phi$ be a continuous level set function. 

We define the domains and the interface

$$
\begin{array}{rccc}
  \text{domain:} & \Omega_{-} := \{ \phi < 0 \}, & \quad
  \Omega_{+} := \{ \phi > 0 \}, & \quad
  \Gamma := \{ \phi = 0 \}. \\
  \texttt{ngsxfem} \text{ keyword:} & \texttt{NEG} & 
  \texttt{POS} & 
  \texttt{IF}  
\end{array}
$$

Let us import `Netgen/NGSolve` stuff first:

In [ ]:
#import netgen.gui
# ngsolve stuff
from ngsolve import *
from ngsolve.webgui import *
# basic geometry features (for the background mesh)
from netgen.geom2d import SplineGeometry
# visualization stuff
from ngsolve.internal import *

We add the basic functionality of the `ngsxfem` package:

In [ ]:
# basic xfem functionality
from xfem import *

## The background mesh
We generate the background mesh of the domain

In [ ]:
square = SplineGeometry()
square.AddRectangle([-1.5,-1.5],[1.5,1.5],bc=1)
mesh = Mesh (square.GenerateMesh(maxh=0.5, quad_dominated=False))

On the background mesh we define the level set function. In the visualization -- using `DrawDC` -- we distinguish the negative and the positive part of the level set values corresponding to the subdomains that are described.

In [ ]:
levelset = (sqrt(sqrt(x**4+y**4)) - 1.0)
DrawDC(levelset, -1.0, 2.0, mesh, "x")
help(DrawDC)

## Geometry approximation through level set approximation
* Arbitrary level set functions can be difficult to handle (cut topology)
* To obtain something feasible, we interpolate into the space of piecewise linear functions
* The thusly obtained approximation is straight within each (simplex) element
* Simplifies numerical integration
* We will discuss enhancements later.

In [ ]:
lsetp1 = GridFunction(H1(mesh,order=1))
InterpolateToP1(levelset,lsetp1)
DrawDC(lsetp1,-1,1,mesh,"lsetp1")

* `InterpolateToP1` takes vertex values (in contrast to `Set`)

## Cut Information

In unfitted FEM it is important to know which elements are cut by the interface $\{ \phi = 0 \}$. These informations are gathered in the cut information class:

In [ ]:
ci = CutInfo(mesh, lsetp1)
help(CutInfo)

### Marking elements

### `DOMAIN_TYPE`
We can ask for BitArrays corresponding to the differently marked elements 

  * `NEG` : True if $\phi < 0$ everywhere on this element, else False
  * `POS` : True if $\phi > 0$ everywhere on this element, else False
  * `IF` : True if $\phi = 0$ somewhere on this element, else False  

In [ ]:
dts = [IF,NEG,POS]
def DrawMarkedElements():
    for dt in dts: 
        print("Drawing", dt); 
        Draw(BitArrayCF(ci.GetElementsOfType(dt)),mesh,"elements_"+str(dt))
        yield
dme = DrawMarkedElements()

In [ ]:
next(dme)

In [ ]:
next(dme)

In [ ]:
next(dme)

### `COMBINED_DOMAIN_TYPE`
There are also some predefined combinations of the previous markers:
  
| IF|POS|NEG| combined flag | description    |
|---|---|---|---------------|----------------|
|  0|  0|  0|            NO | False on all elements
|  0|  0|  1|           NEG | True if $\phi < 0$ everywhere on this element, else False   
|  0|  1|  0|           POS | True if $\phi > 0$ everywhere on this element, else False
|  0|  1|  1|         UNCUT | True if $\phi \neq 0$ everywhere on this element, else False   
|  1|  0|  0|            IF | True if $\phi = 0$ somewhere on this element, else False  
|  1|  0|  1|        HASNEG | True if $\phi \leq 0$ somewhere on this element, else False
|  1|  1|  0|        HASPOS | True if $\phi \geq 0$ somewhere on this element, else False
|  1|  1|  1|           ANY | True on all elements
  

In [ ]:
cdts = [NO,CDOM_NEG,CDOM_POS,UNCUT,CDOM_IF,HASNEG,HASPOS,ANY]
for dt in cdts: print(int(dt),": {0:03b} ,".format(int(dt)),str(dt) )

In [ ]:
for dt in cdts: print(dt);Draw(BitArrayCF(ci.GetElementsOfType(dt)),mesh,"elements_"+str(dt),min=0,max=1)

## Updating the CutInformation
* level set function can be changed with `Update`:

In [ ]:
levelset = sqrt((x-0.5)**2+y**2)-1
scene = Draw(levelset,mesh,"levelset") # overwrite visualization
InterpolateToP1(levelset,lsetp1)
ci.Update(lsetp1);scene.Redraw()

## Marking with facets 
 * `GetFacetsWithNeighborTypes`
 * `GetElementsWithNeighborFacets`

* Sometimes want to mark some facets depending on their neighbor states (NEG/POS/IF)
* Needed for some stabilizations in unfitted FEM (CutFEM/XFEM/..)
* Facet-marking based on two element-based BitArrays. 
* There are two options to connect two BitArrays A and B:
 * use_and=True (default): the result is True for a facet if (A(left) and B(right)) or (B(left) and A(right))
 * use_and=False     (or): the result is True for a facet if (A(left) or B(right)) or (B(left) or A(right))
 
| A(left) | B(right) | partial result (`use_and=True`) | partial result (`use_and=False`) |
|----------|-----------|-----------------------|------------------------|
| False    | False     | False                 | False                  |
| False    | True      | False                 | True                   |
| True     | False     | False                 | True                   |
| True     | True      | True                  | True                   |
 
* with bnd_val_a and bnd_val_b (default: True) we can prescribe how the BitArray should be interpreted if there is only one neighbor (boundary).

### Example 1:

* All facets that are on the outer boundary of the NEG-domain:

In [ ]:
#help(GetFacetsWithNeighborTypes)
ba_facets = GetFacetsWithNeighborTypes(mesh,
                                       a=ci.GetElementsOfType(NEG),
                                       b=ci.GetElementsOfType(IF))

* We can also go from facets to elements

In [ ]:
#help(GetElementsWithNeighborFacets)
ba_surround_facets = GetElementsWithNeighborFacets(mesh,ba_facets)

In [ ]:
def SelectFacetsToDraw(dt1,dt2): 
    ba_facets = GetFacetsWithNeighborTypes(mesh,a=ci.GetElementsOfType(dt1), b=ci.GetElementsOfType(dt2))
    ba_surround_facets = GetElementsWithNeighborFacets(mesh,ba_facets)
    Draw(BitArrayCF(ba_surround_facets), mesh, "surrounding_facets")    

In [ ]:
SelectFacetsToDraw(CDOM_IF,CDOM_NEG)

### Example 2:
We can successively apply this to extend from some part of the domain by direct neighbors, etc .... 

(execute the next cell several times)

In [ ]:
ba_facets = GetFacetsWithNeighborTypes(mesh,a=ba_surround_facets,b=ba_surround_facets,
                                           bnd_val_a=False,bnd_val_b=False,use_and=False)
ba_surround_facets = GetElementsWithNeighborFacets(mesh,ba_facets)
Draw(BitArrayCF(ba_surround_facets), mesh, "ba_surround")

We now learned how to mark elements and facets. Next, we want to apply this to adapt finite element formulations.

## Restriction to *active* mesh/`dof`s

* Unfitted FEM: Solve a PDE problem *restricted* to a subdomain, e.g.
  * a fictitious domain method on $\Omega_{-} = \{ \phi < 0 \}$ or
  * a trace finite element on $\Gamma = \{ \phi = 0\}$
* Standard `FESpace` but only a selection of `dof`s (`active_dofs`)
* We have helper functions for that as well

### Marking dofs (`GetDofsOfElements`)
With `GetDofsOfElements` we can select a subset of dofs corresponding to a set of elements.
With `Compress` we can reduce a finite element space to the corresponding dofs:


In [ ]:
Vhbase = H1(mesh, order=1, dirichlet=[])
VhC = Compress(Vhbase,GetDofsOfElements(Vhbase,ci.GetElementsOfType(HASNEG)))

Alternatively, you can directly `Restrict` a finite element space to a set of elements:

In [ ]:
VhR = Restrict(Vhbase,ci.GetElementsOfType(HASNEG))

Now, we can compare:

In [ ]:
print(Vhbase.ndof, VhC.ndof, VhR.ndof) 

`Compress` and `Restrict` yield the same amount of dofs. However, there is a subtle difference: `VhR` is not defined away from the marked elements whereas `VhC` can be seen as the background `FESpace` with the inactive dofs set to zero. 

Next:

 * `Compress` and `Restrict` allow to *restrict* vectors and matrices to "active" subset of the background FE space
 * What to do if you want to set up the finite element formulation only on a selection of elements / facets / unknowns?

### Marking integrators (`definedonelements`)
We can define integrators on a set of elements only (this is a standard NGSolve feature):

In [ ]:
integrator = VhR.TrialFunction()*VhR.TestFunction() * dx(definedonelements=ci.GetElementsOfType(HASNEG))

In [ ]:
print(ci.GetElementsOfType(HASNEG))

Note that the BitArray taken in the `definedonelements` argument is taken by reference, i.e. later changes to the `CutInfo` `ci` will automatically change the set of elements the `integrator` will act on.

### Restricted Bilinearforms
Furthermore, to avoid that matrices are set up for the full standard FESpace we can restrict matrix couplings to elements (and facets for dg terms). This may be attractive if you do not work with `Restrict`ed or `Compress`ed finite element spaces (e.g. because of changing domains) but want the linear systems to be of reduced size. Further, in unfitted FEM often stabilizations of "Ghost Penalty" type are applied which may lead to dgjumps, i.e. couplings of dofs over facets. To avoid that these couplings are introduced on the whole domain, you may want to restrict the facet couplings explicitly. 

In [ ]:
#help(RestrictedBilinearForm)
a_full = BilinearForm(Vhbase, check_unused=False)
a = RestrictedBilinearForm(Vhbase,element_restriction=ci.GetElementsOfType(IF),
                           facet_restriction=None, check_unused=False)
a_full.Assemble()
a.Assemble()

* This does not change the dimension:

In [ ]:
print(a.mat.height,a.mat.width)
print(a_full.mat.height,a_full.mat.width)

* But the number on non-zero entries:

In [ ]:
print(len(a_full.mat.AsVector()))
print(len(a.mat.AsVector()))